In [38]:
import os
print(os.getcwd())

/home/ec2-user/SageMaker/research_methodology_extraction/src


In [2]:
import sys
print(sys.executable)
print(sys.prefix)

/home/ec2-user/anaconda3/envs/pytorch_p310/bin/python
/home/ec2-user/anaconda3/envs/pytorch_p310


In [5]:
import importlib
with open("/home/ec2-user/SageMaker/research_methodology_extraction/requirements.txt") as reqstxt:
    reqscontent = reqstxt.read()

reqslines = reqscontent.split('\n')
reqs = {}
for reqsline in reqslines:
    module, version = reqsline.split('==')
    reqs[module] = version
    locals()[module] = importlib.import_module(module)
    print(f'Installed {module}.__version__=={locals()[module].__version__}', 'as expected' if locals()[module].__version__==version else f'ERROR: expected {version}')

Installed numexpr.__version__==2.11.0 as expected
Installed pandas.__version__==2.3.1 as expected
Installed sagemaker.__version__==2.250.0 as expected
Installed transformers.__version__==4.55.2 as expected
Installed datasets.__version__==4.0.0 as expected
Installed accelerate.__version__==1.10.0 as expected


In [4]:
# list all directly installed packages but not dependencies
!pip list --not-required

Package                 Version
----------------------- ---------------
accelerate              1.10.0
aiohttp                 3.12.15
Authlib                 1.6.0
awscli                  1.41.13
awscrt                  0.26.1
bokeh                   3.7.3
Brotli                  1.1.0
cached-property         1.5.2
captum                  0.8.0
datasets                4.0.0
distro                  1.8.0
environment-kernels     1.2.0
fqdn                    1.5.1
gmpy2                   2.2.1
gssapi                  1.9.0
gym                     0.26.2
h2                      4.2.0
imageio                 2.37.0
importlib_resources     6.5.2
isoduration             20.11.0
jsonpointer             3.0.0
krb5                    0.7.1
marshmallow             4.0.0
mpi4py                  4.1.0
munkres                 1.1.4
nltk                    3.9.1
numexpr                 2.11.0
nvgpu                   0.10.0
onnx                    1.16.2
opencv-python           4.11.0.86
pickleshare

In [36]:
import json
import boto3
import base64
with open('/etc/opt/ml/sagemaker-notebook-instance-config.json') as json_file:
    notebook_config = json.load(json_file)
print('notebook_config:', notebook_config)
print("notebook_config['notebook_uri']:", notebook_config['notebook_uri'])
# Not using regex on purpose. Parsing an already double-escaped regex pattern is doable, but is usually very brittle. 
# In this case it's more robust to build the pattern matching on simple splitting based on structures we can assume an automatically generated configuration string always has.
parts_first = notebook_config['notebook_uri'].split('(')
print('parts_first:', parts_first)
parts_second = parts_first[1].split('|')
print('parts_second:', parts_second)
notebook_instance_name = parts_second[0]
print(f'notebook_instance_name: {notebook_instance_name}')

boto_sagemaker = boto3.client('sagemaker')
notebok_instance_description = boto_sagemaker.describe_notebook_instance(
    NotebookInstanceName = notebook_instance_name
)
print('notebok_instance_description: ', notebok_instance_description)
notebok_instance_lifecycle_config_name = notebok_instance_description['NotebookInstanceLifecycleConfigName']
print('notebok_instance_lifecycle_config_name:', notebok_instance_lifecycle_config_name)
notebok_instance_lifecycle_config_description = boto_sagemaker.describe_notebook_instance_lifecycle_config(
    NotebookInstanceLifecycleConfigName = notebok_instance_lifecycle_config_name
)
print('notebok_instance_lifecycle_config_description_', notebok_instance_lifecycle_config_description)
lifecycle_on_create_base64 = notebok_instance_lifecycle_config_description['OnCreate'][0]['Content']
lifecycle_on_start_base64 = notebok_instance_lifecycle_config_description['OnStart'][0]['Content']
print('lifecycle_on_create_base64:', lifecycle_on_create_base64)
print('lifecycle_on_start_base64:', lifecycle_on_start_base64)
lifecycle_on_create_string = '\n'.join(f'{base64.b64decode(lifecycle_on_create_base64)}'.split('\\n'))
lifecycle_on_start_string = '\n'.join(f'{base64.b64decode(lifecycle_on_start_base64)}'.split('\\n'))
print('lifecycle_on_create_string:', '\n', lifecycle_on_create_string)
print('lifecycle_on_start_string:', '\n', lifecycle_on_start_string)

notebook_config: {'csrf_check': 'enabled', 'direct_internet_access': 'enabled', 'vpc_support': 'disabled', 'notebook_uri': 'https:\\/\\/(test017-ml-g5-xlarge|test017-ml-g5-xlarge).notebook.eu-west-2.sagemaker.aws', 'root_access': 'enabled'}
notebook_config['notebook_uri']: https:\/\/(test017-ml-g5-xlarge|test017-ml-g5-xlarge).notebook.eu-west-2.sagemaker.aws
parts_first: ['https:\\/\\/', 'test017-ml-g5-xlarge|test017-ml-g5-xlarge).notebook.eu-west-2.sagemaker.aws']
parts_second: ['test017-ml-g5-xlarge', 'test017-ml-g5-xlarge).notebook.eu-west-2.sagemaker.aws']
notebook_instance_name: test017-ml-g5-xlarge
notebok_instance_description:  {'NotebookInstanceArn': 'arn:aws:sagemaker:eu-west-2:762595428873:notebook-instance/test017-ml-g5-xlarge', 'NotebookInstanceName': 'test017-ml-g5-xlarge', 'NotebookInstanceStatus': 'InService', 'Url': 'test017-ml-g5-xlarge.notebook.eu-west-2.sagemaker.aws', 'InstanceType': 'ml.g5.xlarge', 'RoleArn': 'arn:aws:iam::762595428873:role/SageMakerExecutionRole',